# Index constituent and Ownership

In [1]:
library(DBI)
library(plutoDbR)
library(plutoR)
library(tidyverse)
options("scipen"=999)
options(stringsAsFactors = FALSE)
source("/usr/share/pluto/config.R")
source("/usr/share/pluto/goofy/plot.common.R")
source("/usr/share/pluto/goofy/misc.common.R")

library(pracma)
library(ggthemes)
library(reshape2)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)
library(ggpubr)

options(repr.plot.width=16, repr.plot.height=8)
options(tibble.width = Inf)

equities <- EquitiesIndiaNse()
indices <- Indices()

In [2]:
getShpNew <- function(sym){
    maxDt <- equities$ShareholdingPatternXbrl() %>%
        filter(SYMBOL == sym) %>%
        summarize(M = max(TIME_STAMP)) %>%
        collect()
    
    maxDt <- as.Date(maxDt$M[1])
    
    PI <- equities$ShareholdingPatternXbrl() %>%
        filter(SYMBOL == sym & TIME_STAMP == maxDt 
               & KEY_2 == "ShareholdingAsAPercentageOfTotalNumberOfShares"
               & (KEY_1 == "ShareholdingOfPromoterAndPromoterGroup"
                  || KEY_1 == "IndividualShareholdersHoldingNominalShareCapitalUpToRsTwoLakh"
                  || KEY_1 == "IndividualShareholdersHoldingNominalShareCapitalInExcessOfRsTwoLakh"
                  || KEY_1 == "IndividualsOrHinduUndividedFamily")) %>%
        summarize(T = sum(VALUE)) %>%
        collect()
    
    MF <- equities$ShareholdingPatternXbrl() %>%
        filter(SYMBOL == sym & TIME_STAMP == maxDt 
               & KEY_2 == "ShareholdingAsAPercentageOfTotalNumberOfShares"
               & KEY_1 == "MutualFundsOrUti") %>%
        summarize(T = sum(VALUE)) %>%
        collect()
    
    return (c(toString(maxDt), as.numeric(PI$T[1]), as.numeric(MF$T[1])))
}

In [12]:
getShpOld <- function(sym){
    minDt <- equities$ShareholdingPatternRaw() %>%
        filter(SYMBOL == sym 
               & KEY %like% "%|a|%"
               & KEY %like% "%fund%"
               & KEY %like% "%SCRR%") %>%
        summarize(M = min(TIME_STAMP)) %>%
        collect()
    
    minDt <- as.Date(minDt$M[1])
    
    PROMOTER <- equities$ShareholdingPatternRaw() %>%
        filter(SYMBOL == sym & TIME_STAMP == minDt 
               & KEY %like% "%a|%"
               & KEY %like% "%promoter%"
               & KEY %like% "%SCRR%") %>%
        select(VALUE) %>%
        collect()
    
    HUF <- equities$ShareholdingPatternRaw() %>%
        filter(SYMBOL == sym & TIME_STAMP == minDt 
               & KEY %like% "%hindu%"
               & KEY %like% "%SCRR%") %>%
        select(VALUE) %>%
        collect()
    
    INDI <- equities$ShareholdingPatternRaw() %>%
        filter(SYMBOL == sym & TIME_STAMP == minDt 
               & KEY %like% "%nominal%capital%"
               & KEY %like% "%SCRR%") %>%
        select(VALUE) %>%
        collect()
    
    MF <- equities$ShareholdingPatternRaw() %>%
        filter(SYMBOL == sym & TIME_STAMP == minDt 
               & KEY %like% "%|a|%"
               & KEY %like% "%fund%"
               & KEY %like% "%SCRR%") %>%
        select(VALUE) %>%
        collect()
    
    return (c(toString(minDt), as.numeric(PROMOTER$VALUE[1]) + as.numeric(HUF$VALUE[1]) + as.numeric(INDI$VALUE[1]), as.numeric(MF$VALUE[1])))
}

In [4]:
indexShpNew <- function(indexName){
    print(indexName)
    constitutents <- indices$NseConstituents() %>%
        filter(NAME == indexName) %>%
        select(SYMBOL, CAP_WEIGHT) %>%
        collect() %>%
        as.data.frame()

    constitutents$PI <- NA
    constitutents$MF <- NA

    for (i in 1:nrow(constitutents)){
        symbol <- toString(constitutents$SYMBOL[i])
        tryCatch({
                pimf <- getShpNew(symbol)
                constitutents$PI[i] <- pimf[1]
                constitutents$MF[i] <- pimf[2]
            },
            error = function(e){
                print(symbol)
                print(e)
            })
    }

    constitutents$INST <- 100 - (constitutents$PI + constitutents$MF)
    
    constitutents$MF_WT <- constitutents$MF * constitutents$CAP_WEIGHT/100
    constitutents$INST_WT <- constitutents$INST * constitutents$CAP_WEIGHT/100
    
    print("Mutual fund ownership (%)")
    print(summary(constitutents$MF))
    
    print("Institutional ownership (%)")
    print(summary(constitutents$INST))
    
    print("Total MF ownership (%)")
    print(sum(constitutents$MF_WT, na.rm=T))
    
    print("Total Institutional ownership (%)")
    print(sum(constitutents$INST_WT, na.rm=T))
}

In [9]:
indexShpOldNew <- function(indexName){
    print(indexName)
    constitutents <- indices$NseConstituents() %>%
        filter(NAME == indexName) %>%
        select(SYMBOL, CAP_WEIGHT) %>%
        collect() %>%
        as.data.frame()

    constitutents$ASOF_OLD <- NA
    constitutents$PI_OLD <- NA
    constitutents$MF_OLD <- NA
    
    constitutents$ASOF_NEW <- NA
    constitutents$PI_NEW <- NA
    constitutents$MF_NEW <- NA

    for (i in 1:nrow(constitutents)){
        symbol <- toString(constitutents$SYMBOL[i])
        tryCatch({
                pimf <- getShpOld(symbol)
                constitutents$ASOF_OLD[i] <- pimf[1]
                constitutents$PI_OLD[i] <- as.numeric(pimf[2])
                constitutents$MF_OLD[i] <- as.numeric(pimf[3])
            },
            error = function(e){
                print(symbol)
                print(e)
            })
        
        tryCatch({
                pimf <- getShpNew(symbol)
                constitutents$ASOF_NEW[i] <- pimf[1]
                constitutents$PI_NEW[i] <- as.numeric(pimf[2])
                constitutents$MF_NEW[i] <- as.numeric(pimf[3])
            },
            error = function(e){
                print(symbol)
                print(e)
            })
    }
    
    constitutents$INST_OLD <- 100 - (constitutents$PI_OLD + constitutents$MF_OLD)
    
    constitutents$MF_WT_OLD <- constitutents$MF_OLD * constitutents$CAP_WEIGHT/100
    constitutents$INST_WT_OLD <- constitutents$INST_OLD * constitutents$CAP_WEIGHT/100

    constitutents$INST_NEW <- 100 - (constitutents$PI_NEW + constitutents$MF_NEW)
    
    constitutents$MF_WT_NEW <- constitutents$MF_NEW * constitutents$CAP_WEIGHT/100
    constitutents$INST_WT_NEW <- constitutents$INST_NEW * constitutents$CAP_WEIGHT/100
    
    return(constitutents)
}

In [19]:
shpData <- indexShpOldNew("NIFTY 100")

[1] "NIFTY 100"


[1] "ITC"
<simpleError in as.Date.default(x, ...): do not know how to convert 'x' to class “Date”>


In [20]:
print(shpData)

        SYMBOL CAP_WEIGHT   ASOF_OLD PI_OLD MF_OLD   ASOF_NEW PI_NEW MF_NEW
1          ACC     0.2258 2015-12-31  61.30   3.68 2019-09-30  64.58  12.33
2   ADANIPORTS     0.5336 2015-12-31  58.91   2.88 2019-09-30  64.73   2.24
3    AMBUJACEM     0.2608 2015-12-31  57.11   3.73 2019-09-30  67.89   4.77
4     ASHOKLEY     0.2002 2015-12-31  59.91   3.14 2019-09-30  65.34   7.25
5   ASIANPAINT     1.4123 2015-12-31  76.20   1.89 2019-09-30  75.19   5.21
6   AUROPHARMA     0.2226 2015-12-31 107.66   6.36 2019-09-30  72.75  12.66
7     AXISBANK     2.8862 2015-12-31  36.67  10.76 2019-09-30  21.55  18.27
8   BAJAJ-AUTO     0.7382 2015-12-31  59.70   1.20 2019-09-30  73.62   2.69
9   BAJAJFINSV     0.9330 2015-12-31  72.46   5.85 2019-09-30  82.34   3.56
10  BAJAJHLDNG     0.3358 2015-12-31  68.41   1.33 2019-09-30  72.51   1.23
11  BAJFINANCE     1.7335 2015-12-31  66.31   6.67 2019-11-07  64.85   7.43
12  BANDHANBNK     0.2524 2018-03-26  85.66   1.43 2019-10-21  66.01   3.83
13  BANKBARO

### Before

In [21]:
print("Mutual fund ownership (%)")
print(summary(shpData$MF_OLD))
    
print("Institutional ownership (%)")
print(summary(shpData$INST_OLD))
    
print("Total MF ownership (%)")
print(sum(shpData$MF_WT_OLD, na.rm=T))
    
print("Total Institutional ownership (%)")
print(sum(shpData$INST_WT_OLD, na.rm=T))

[1] "Mutual fund ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.100   1.538   3.405   4.302   5.902  18.030 
[1] "Institutional ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -42.21   18.37   28.14   29.76   39.12   89.56 
[1] "Total MF ownership (%)"
[1] 6.152912
[1] "Total Institutional ownership (%)"
[1] 44.42422


### Now

In [22]:
print("Mutual fund ownership (%)")
print(summary(shpData$MF_NEW))
    
print("Institutional ownership (%)")
print(summary(shpData$INST_NEW))
    
print("Total MF ownership (%)")
print(sum(shpData$MF_WT_NEW, na.rm=T))
    
print("Total Institutional ownership (%)")
print(sum(shpData$INST_WT_NEW, na.rm=T))

[1] "Mutual fund ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  0.320   3.365   5.630   6.882   9.410  29.660       1 
[1] "Institutional ownership (%)"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 -44.42   19.13   27.41   28.27   36.35   82.63       1 
[1] "Total MF ownership (%)"
[1] 9.505658
[1] "Total Institutional ownership (%)"
[1] 37.81306


This notebook was created using [pluto](https://pluto.studio). Thank you for playing fair!